In [2]:
n_threads = 4
from multiprocessing import Process, Lock, Pool
def f(l, i):
    l.acquire()
    print 'hello world', i
    l.release()

lock = Lock()
for num in range(n_threads):
    Process(target=f, args=(lock, num)).start()

hello world 0
hello world 1
hello world 2
hello world 3


In [13]:
n_threads = 4
from multiprocessing import Process, Lock, Manager

result = []
counter = 0

def f(l, n):
    global result, counter
    
    o = range(10*n, 10*n + 10)
    l.acquire()
    print n, "|", o
    result += o
    l.release()

    
manager = Manager()
lock = Lock()
for num in range(n_threads):
    p = Process(target=f, args=(lock, num)).start()
    
print result

0 | [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]
1 | [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
2 | [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
3 | [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [14]:
n_threads = 4
from threading import Thread, Lock

result = []
counter = 0

def f(l, n):
    global result, counter
    
    o = range(10*n, 10*n + 10)
    l.acquire()
    print n, "|", o
    result += o
    l.release()

    
manager = Manager()
lock = Lock()
for num in range(n_threads):
    t = Thread(target=f, args=(lock, num)).start()
    
print result

0 | [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1 | [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
2 | [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
3 | [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
